# A Simple CSV Loader

In [ ]:
%matplotlib inline

import dg.conn
con = dg.conn.Conn(user="ftian", port=5555)
print (con.ver)

In [ ]:
from dg.dsutil.csv import CsvXt

In [ ]:
# Iris data set is probably the single most used dataset.
# From the sepal/petal lengh/width, can you guess the iris class?
csv = CsvXt('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data') 
csv.add_col('sepal_length', 'float')
csv.add_col('sepal_width', 'float') 
csv.add_col('petal_length', 'float') 
csv.add_col('petal_width', 'float') 
csv.add_col('iris_class', 'text')
xt = csv.xtable(con)

In [ ]:
# I don't know how, but I can check some basics.  At least let's see if the
# avg is significant.   Amateur's first take.   Hmm, Let me guess -- 
# petal size will be enough to tell setosa apart, but virginica
# and versicolor will be harder 
import dg.xtable
avgt = dg.xtable.fromQuery(con, """
    select iris_class, avg(sepal_length) as asl, avg(sepal_width) as asd,
                       avg(petal_length) as apl, avg(petal_width) as apd
    from #0# 
    group by iris_class
""", inputs=[xt])
print(avgt.show())

In [ ]:
# Ever execution of the xtable cause internet traffic.
# CTAS.
csv.ctas(con, "iris", distributed_by='iris_class')

In [ ]:
localtab = dg.xtable.fromTable(con, 'iris')
print(localtab.show(tablefmt='fancy_grid'))